Easom Table Function 
=====================
It is not centered at $(0,0)$. 
$$
f(x) = -\cos(x[1]) * \cos(x[2]) * \exp(-((x[1] - \pi)^2 + (x[2] - \pi)^2))
$$
 is defined over the square $[-100, 100]^2$

In [17]:
using Pkg
Pkg.activate("../../.")
using Globtim
# include("../../../src/lib_func.jl") # Include the library of functions

# Constants and Parameters
d = 2 # Initial Degree 
const n, a, b = 2, 2, 1 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .5 , 1 / 10  # Sampling parameters
const tol_l2 = 1e-5            # Define the tolerance for the L2-norm

f = easom; # Objective function

  Activating project at `~/Globtim.jl`


In [18]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        println("Current degree: ", d)
        global d += 1
    end
end;

current L2-norm: 0.00014160670023366046
Number of samples: 8
Current degree: 2
current L2-norm: 7.990893716471537e-5
Number of samples: 12
Current degree: 3
current L2-norm: 4.561172282860882e-5
Number of samples: 17
Current degree: 4
current L2-norm: 2.6929142286800935e-5
Number of samples: 22
Current degree: 5
current L2-norm: 1.508777146905729e-5
Number of samples: 28
Current degree: 6
attained the desired L2-norm: 8.20662008547841e-6


In [19]:
using DynamicPolynomials, DataFrames
using HomotopyContinuation, ProgressLogging

@polyvar(x[1:n]) # Define polynomial ring 
pol = main_nd(x, n, d, poly_approx.coeffs) 
grad = differentiate.(pol, x)
sys = System(grad)
println("The system is of degree: ", d-1)

The system is of degree: 6


In [20]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false);

Tracking 36 paths...   6%|█▋                            |  ETA: 0:01:26



Tracking 36 paths... 100%|██████████████████████████████| Time: 0:00:05
  # paths tracked:                  36
  # non-singular solutions (real):  36 (14)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         36 (14)


In [21]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

In [23]:
using GLMakie

# Extract coordinates and function values
coords = poly_approx.scale_factor * poly_approx.grid
z_coords = poly_approx.z

if size(coords)[2] == 2  # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Mixture of Gaussians Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with a darker core
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=:black, label="Sampled Data Core")

    # Plot the sampled data with adjusted size and color to create a halo effect
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=4, color=z_coords, colormap=:viridis, label="Sampled Data Halo")

    # Plot the exact approximant critical points with adjusted size
    scatter!(ax, df.x, df.y, df.z, markersize=10, color=:red, label="Exact approximant critical points")

    display(fig)
end

GLMakie.Screen(...)

In [8]:

coords = poly_approx.scale_factor * poly_approx.grid # Extract coordinates and function values
z_coords = poly_approx.z

if size(coords)[2] == 2 # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Mixture of Gaussians Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with darker contrast
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=:black, colormap=:viridis, label="Sampled Data")
    # Adjust the size and color of the sampled data points
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=5, color=z_coords, colormap=:viridis, label="Sampled Data")

    display(fig)
end

GLMakie.Screen(...)

In [24]:

# Create a grid for the surface plot
x1 = range(-1, 1, length=100)
y = range(-1, 1, length=100)
z = [f([xi, yi]) for xi in x1, yi in y]

fig = Figure(size=(800, 600))
ax = Axis3(fig[1, 1], title="3D Plot of Shubert function with the critical points of the approximant of degree 22",
           xlabel="x", ylabel="y", zlabel="f(x, y)")

# Plot the surface
surface!(ax, x1, y, z, color=(:white, 0.25), transparency=true)
# map=:(viridis,.)

# Plot the critical points
scatter!(ax, df.x, df.y, df.z, markersize=5, color=:red, label="Critical Points")

fig

Should plot the polynomial approximant too.

In [ ]:
# Update tolerance
const tol_l2 = 1.5e-3

# Re-run the main computation loop
d = 2 # Reset degree
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        println("Current degree: ", d)
        global d += 1
    end
end;

# Re-run the polynomial system setup and solve
pol = main_nd(x, n, d, poly_approx.coeffs) 
grad = differentiate.(pol, x)
sys = System(grad)
println("The system is of degree: ", d-1)

Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false)

# Filter points and create DataFrame
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Collect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

current L2-norm: 0.03150710328000877


Number of samples: 8
Current degree: 2
current L2-norm: 0.023311707351089422
Number of samples: 12
Current degree: 3
current L2-norm: 0.014570058361619327
Number of samples: 17
Current degree: 4
current L2-norm: 0.010413345695311693
Number of samples: 22
Current degree: 5
current L2-norm: 0.007463075052903172
Number of samples: 28
Current degree: 6
current L2-norm: 0.005525057615807029
Number of samples: 34
Current degree: 7
current L2-norm: 0.003966811072142175
Number of samples: 41
Current degree: 8
current L2-norm: 0.0027412849478182844
Number of samples: 48
Current degree: 9
current L2-norm: 0.002238521327513709
Number of samples: 56
Current degree: 10
current L2-norm: 0.0016164080957956177
Number of samples: 65
Current degree: 11
current L2-norm: 0.0013581351222098238
Number of samples: 74
Current degree: 12
current L2-norm: 0.00108357704548822
Number of samples: 83
Current degree: 13
current L2-norm: 0.0009146586221214484
Number of samples: 93
Current degree: 14
current L2-norm:

In [8]:
# Plot using GLMakie

# Extract coordinates and function values
coords = poly_approx.scale_factor * poly_approx.grid
z_coords = poly_approx.z

if size(coords)[2] == 2  # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Mixture of Gaussians Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with adjusted size and color to create a halo effect
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=z_coords, colormap=:viridis, label="Sampled Data Halo")

    # Plot the exact approximant critical points with adjusted size
    scatter!(ax, df.x, df.y, df.z, markersize=10, color=:red, label="Exact approximant critical points")

    display(fig)
end

GLMakie.Screen(...)